In [3]:
import os
from dotenv import load_dotenv
from serpapi import GoogleSearch
load_dotenv()
serpapi_key = os.getenv("SERPAPI_KEY")

In [19]:
"""Create Search Parameter for grocery stores within greater Rochester area

"""
#key

# Latitude and Longitude of Rochester NY
roc_lat = "43.161030"
roc_long = "-77.610924"
ROC_NY = [roc_lat, roc_long]

query = "Grocery Store"
params = {
    "engine": "google_maps",
    "q": query,
    "location":"43.161030 , -77.610924",
    "radius": 16093.44,
    "api_key":serpapi_key,
    "start":0,
    "ll":"@43.161030 , -77.610924,21z"

}

In [20]:
search = GoogleSearch(params)
results = search.get_dict()

In [21]:
print(results)

{'search_metadata': {'id': '675a4947f61bb8e849663ddf', 'status': 'Success', 'json_endpoint': 'https://serpapi.com/searches/6bf48ce5fd471eb5/675a4947f61bb8e849663ddf.json', 'created_at': '2024-12-12 02:24:07 UTC', 'processed_at': '2024-12-12 02:24:07 UTC', 'google_maps_url': 'https://www.google.com/search?q=Grocery+Store&hl=en&tbm=map&tch=1&pb=!4m9!1m3!1d181.8943554610914!2d-77.610924!3d43.161030!2m0!3m2!1i784!2i644!4f13.1!7i20!8i0!10b1!12m8!1m1!18b1!2m3!5m1!6e2!20e3!10b1!16b1!19m4!2m3!1i360!2i120!4i8!20m57!2m2!1i203!2i100!3m2!2i4!5b1!6m6!1m2!1i86!2i86!1m2!1i408!2i240!7m42!1m3!1e1!2b0!3e3!1m3!1e2!2b1!3e2!1m3!1e2!2b0!3e3!1m3!1e3!2b0!3e3!1m3!1e8!2b0!3e3!1m3!1e3!2b1!3e2!1m3!1e9!2b1!3e2!1m3!1e10!2b0!3e3!1m3!1e10!2b1!3e2!1m3!1e10!2b0!3e4!2b1!4b1!9b0!22m6!1sg3qzXsG-JpeGoATHyYKQBw%3A1!2zMWk6MSx0OjExODg3LGU6MCxwOmczcXpYc0ctSnBlR29BVEh5WUtRQnc6MQ!7e81!12e3!17sg3qzXsG-JpeGoATHyYKQBw%3A110!18e15!24m46!1m12!13m6!2b1!3b1!4b1!6i1!8b1!9b1!18m4!3b1!4b1!5b1!6b1!2b1!5m5!2b1!3b1!5b1!6b1!7b1!10m1!8e3!14m1!

In [22]:
import json

# Write the dictionary to a JSON file
with open('data_2.json', 'w') as outfile:
    json.dump(results, outfile, indent=4)
